In [6]:
pip install fastapi uvicorn openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00


In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import openai
from fastapi.middleware.cors import CORSMiddleware
import os

def get_openai_api_key():
    return os.getenv("OPENAI_API_KEY")

app = FastAPI()

# CORS to allow frontend requests
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class UserQuery(BaseModel):
    url: str
    question: str

@app.post("/chat")
def chat_with_gpt(query: UserQuery):
    openai_api_key = get_openai_api_key()
    if not openai_api_key:
        raise HTTPException(status_code=500, detail="Missing OpenAI API Key")

    try:
        prompt = (
            f"A student is struggling with a problem on LeetCode: {query.url}.\n"
            f"They asked: {query.question}.\n"
            "Provide hints and guidance without giving the full solution.\n"
            "Encourage them to think about problem constraints, edge cases, and similar examples."
        )

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful DSA teaching assistant."},
                {"role": "user", "content": prompt},
            ],
            api_key=openai_api_key
        )

        return {"response": response["choices"][0]["message"]["content"]}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

Set your OpenAI API key in an .env file or environment variable:

In [ ]:
export OPENAI_API_KEY="your-api-key"
